# FP - TRUN

In [11]:
from datetime import datetime
import mRun as mr
import utils_data as md
import numpy as np
import pandas as pd

def get_models(type):
    if type == "FRFLO":
        return [
            { 'dt':'C2',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
            { 'dt':'C4',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
            { 'dt':'C1',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
        ]
    elif type == "FRALL1":
        return [
            { 'dt':'C2',  "e":40,  "lr":0.001, "h":[100 , 100], "spn": 40000, "pe": [], "pt": []  },
            { 'dt':'C4',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 40000, "pe": [], "pt": []  },
            { 'dt':'C1',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 40000, "pe": [], "pt": []  },
            # { 'dt':'C0',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
        ]
    else: return []

In [2]:
print("___Start!___" +  datetime.now().strftime('%H:%M:%S')  )
final = "_" ;  md.DESC = "FRALL1";  # FRFLO   FRALL1
ALL_DS = md.LOGDAT + md.DESC + md.DSC 

# DATA READ  ------------------------------------------------ 
# md.mainRead2(ALL_DS, 1, 2 ) # , all = True, shuffle = True  ) 
md.mainRead2(path=ALL_DS, part=1, batch_size=2 ) # For testing I am forced to used JSON - column names and order may be different! 

url_test = md.LOGDAT + "FREXP1/" ; # url_test = "url"
force = False; excel = True  # dataFile = "frall2_json.txt"; labelFile = "datal.csv"     
md.get_tests(url_test, force, excel )


___Start!___15:27:10
data read - 45822 - time:41.2051055431366


In [12]:
# OPERATIONS  ------------------------------------------------ 
# md.get_columns(force)
execc = get_models(md.DESC)
for ex in execc:
    md.spn = ex["spn"]; md.dType = ex["dt"]; mr.epochs = ex["e"]; mr.lr = ex["lr"]; mr.h = ex["h"]

    md.normalize()  
    mr.ninp, mr.nout, mr.top_k = md.getnn()
    md.MODEL_DIR = md.LOGDIR + md.DESC + '/'   + mr.get_hpar(mr.epochs, final=final) +"/" 
    mr.model_path = md.MODEL_DIR + "model.ckpt" 
    mr.build_network3()                                                                                                                                                                                                                                                                                    
    print(mr.model_path)    
    # ex["pe"] = mr.evaluate( )
    ex["pt"] = mr.tests(url_test, p_col=False  )

# PRINTING  ------------------------------------------------ 
print("end!___" +  datetime.now().strftime('%H:%M:%S')  )

build network
../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep40_/model.ckpt
_____TESTS...
Model restored from file: ../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep40_/model.ckpt
INFO:tensorflow:Restoring parameters from ../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep40_/model.ckpt
test ac = 0.9924027
0 RealVal: 1 - 1 - PP: [1 0] PR: [9.999932e-01 6.795939e-06]
1 RealVal: 1 - 1 - PP: [1 0] PR: [9.999994e-01 5.993536e-07]
2 RealVal: 1 - 1 - PP: [1 0] PR: [9.992847e-01 7.153824e-04]
3 RealVal: 1 - 1 - PP: [1 0] PR: [9.993394e-01 6.605258e-04]
4 RealVal: 1 - 1 - PP: [1 0] PR: [9.9999976e-01 2.4944737e-07]
5 RealVal: 1 - 1 - PP: [1 0] PR: [9.9985468e-01 1.4534927e-04]
6 RealVal: 1 - 1 - PP: [1 0] PR: [0.9983192  0.00168074]
7 RealVal: 1 - 1 - PP: [1 0] PR: [1.0000000e+00 2.7529797e-15]
8 RealVal: 1 - 1 - PP: [1 0] PR: [0.98087776 0.01912223]
9 RealVal: 1 - 1 - PP: [1 0] PR: [9.9994147e-01 5.8517275e-05]
10 RealVal: 1 - 1 - PP: [1 0] PR: [9.9988556e-01 1.1440349e-04]
11 RealVal: 1 -

In [18]:
def record_data( ex, dsp, dspo, type = "pt"  ):
    # print(len(np.array([str(xi[0]) for xi in ex["pt"][1]])));     print(len(tsd.columns))
    # tsd.insert(len(tsd.columns), ex["dt"] + 'FP_P', dsp2["FP_P"].map(lambda x: dc( x ) )    )
    #tsd[ "_PRED"] = np.array([str(xi[0]) for xi in ex["pt"][1]])  # pandas warnings! 
    dsp.insert(len(tsd.columns), ex["dt"] + '_FP_P', dspo["FP_P"].map(lambda x: md.dc( x ) ))    
    dsp.insert(len(dsp.columns), ex["dt"] + '_PRDU', np.array([int(xi[0])  for xi in ex[type][1]])  )    
    dsp.insert(len(dsp.columns), ex["dt"] + '_PRED', np.array([str(xi)      for xi in ex[type][1]])  )    
    dsp.insert(len(dsp.columns), ex["dt"] + '_PROB', np.array([str(xi)      for xi in ex[type][0]])  )    

    # error calc: 
    # dsp[ex["dt"] + '_ERR'] = dsp[[ex["dt"] + '_PREDU']] == dsp[[ex["dt"] + '_FP_P']] 
    if ex["dt"] == "C1":
        dsp.insert(len(tsd.columns), ex["dt"] + '_ERR', 
                  #( dsp.loc[:, ex["dt"] + '_FP_P'] == dsp.loc[:,ex["dt"] +'_PRDU']  )  
                   np.array([ abs(dsp.loc[i,ex["dt"] +'_PRDU']-dsp.loc[i,ex["dt"] +'_FP_P'])>3     
                             for i in range(len(dsp))  ]) 
                  )   
    else:
        dsp.insert(len(tsd.columns), ex["dt"] + '_ERR', ( dsp.loc[:, ex["dt"] + '_FP_P'] == dsp.loc[:,ex["dt"] +'_PRDU']  )  )   
    
    return dsp 

tsd = md.dsp[["M", "FP"]]
# for ex in execc:
ex = execc[2]
tsd = record_data( ex, tsd, md.dsp,  type = "pt")    
tsd.to_csv(md.LOGDAT + "testDSJ.csv")
print(tsd.head())

# testing
# if(tsd[["C2_PREDU"]] == tsd[["C2_FP_P"]]): print("HI")
# tsd[["C2_PREDU"]]
# print(tsd.iloc[2])

# tsd.iloc[:,"C2_FP_P"] == tsd.iloc[:,"C2_PRED_U"]
print( (tsd.iloc[:,2] == tsd.iloc[:,3]).sum() )
print(len(tsd))
# tsd.iloc[:,2]
# tsd.loc[:,'C2_FP_P'] == tsd.loc[:,'C2_PREDU']


            M  FP  C1_FP_P  C1_PRDU                C1_PRED  \
0      640309  70       70       74       [74 73 75 76 72]   
1      640310  74       74       76       [76 75 77 74 73]   
2      640391  67       67       70       [70 68 71 69 72]   
3      640411  67       67      101  [101  70  72  73  74]   
4  PAV10150AP  97       97       99  [ 99  97  98  96 101]   

                                             C1_PROB  C1_ERR  
0  [0.1822734  0.1648964  0.14986096 0.13269566 0...    True  
1  [0.23058562 0.18571258 0.15724681 0.14261925 0...   False  
2  [0.19589786 0.1838908  0.16818821 0.15685314 0...   False  
3  [0.14092173 0.14012368 0.11914673 0.10563724 0...    True  
4  [0.17064801 0.16572432 0.15743396 0.13412257 0...   False  
331
1053


# test 

In [4]:
#save copy
# md.normalize()  = apply inverse normalization!
tsd = md.dsp[["M", "FP"]]
tsd.insert(len(tsd.columns), ex["dt"] + 'FP_P', md.dsp["FP_P"].map(lambda x: md.dc( x ) ))    

# for ex in execc:
def record_d( ex, dsp, type = "pt"  ):

    print(len(np.array([str(xi[0]) for xi in ex["pt"][1]])))
    print(len(tsd.columns))
    
    tsd.insert(len(tsd.columns), ex["dt"] + 'FP_P', dsp["FP_P"].map(lambda x: dc( x ) )    )
    
        
    #tsd[ "_PRED"] = np.array([str(xi[0]) for xi in ex["pt"][1]])  
    #tsd.insert(len(tsd.columns), ex["dt"] + '_PRED', np.array([str(xi[0]) for xi in ex["pt"][1]])  )    
    tsd.insert(len(tsd.columns), ex["dt"] + '_PREDU', np.array([str(xi[0]) for xi in ex["pt"][1]])  )    
    tsd.insert(len(tsd.columns), ex["dt"] + '_PRED', np.array([str(xi) for xi in ex["pt"][1]])  )    
    tsd.insert(len(tsd.columns), ex["dt"] + '_PROB', np.array([str(xi) for xi in ex["pt"][0]])  )    

    # error calc: 
    #tsd[ex["dt"] + '_ERR'] = tsd.[[ex["dt"] + '_PREDU']] == 
    
    # tsd["PRED"] = ex["pt"][1][i][0] # pred 1
    # tsd[ex["dt"] + "_PRED"] = ex["pt"][1] # pred 1
    # tsd[ex["dt"] + "_PROB"] = ex["pt"][0] # prob 

    #tsd.insert(2, '_PRED', np.array([str(xi[0]) for xi in ex["pt"][1]])  )    
    #tsd[ex["dt"] + "_PROB"] = ex["pt"][0] # prob 
    
#tsd.to_csv(md.LOGDAT + "testDSJ.csv")
tsd.head()
# md.dsp.head()


,M,FP,C2FP_P
0,640309,70,1
1,640310,74,1
2,640391,67,1
3,640411,67,1
4,PAV10150AP,97,1


# Print formula 

In [22]:
# md.print_form2(md.dsp.iloc[0])
# PBV11270AD
# md.print_form2(md.dst[md.dst["M"]==100456].iloc[0])
md.print_form2(md.dsp[md.dsp["M"]=='PBV11270AD'].iloc[0])

astr = md.get_components(md.dsp.iloc[0])
# print(astr)

def print_list(astr, name = "/_logs_tr3" ):
    file = md.LOGDAT +  name + ".txt" # + md.DESC +
    f = open(file, 'w')
    for line in astr:
        f.write(line + "\n") 
    f.close()       #close file
    
print_list(astr)

N. of components: 48
components:____ tot ______FP_____values
M       -      (    )--(     ) - PBV11270AD
FP      -      (    )--(     ) -         59
100419  -      ( 176)--( 40.0) -  4.4444446% 		X
103011  -      (6116)--( 72.0) -  8.3333336% 		
103090  -      ( 186)--( 89.0) -  0.6666667% 		
103156  -      (2180)--(101.0) - 1.1111111000000002% 		
103247  -      (3316)--(101.0) - 0.5555555999999999% 		
103605  -      ( 960)--( 98.0) - 0.5555555999999999% 		
103797  -      (2411)--( 52.0) - 3.8888890000000003% 		X
103798  -      (3151)--( 82.0) - 0.11111110000000002% 		
103799  -      (4944)--( 87.0) - 13.888889599999999% 		
103933  -      ( 527)--( 88.0) - 0.055555600000000004% 		
105169  -      (2496)--( 41.0) -  0.3333333% 		X
105461  -      ( 584)--( 78.0) - 0.5555555999999999% 		
113554  -      (5645)--(151.0) -      0.066% 		
130077  -      (3131)--(116.0) - 0.055555600000000004% 		
130337  -      ( 155)--( 85.0) - 0.055555600000000004% 		
130659  -      ( 695)--(134.0) - 1.111111